In [1]:
import nibabel as nib
from pathlib import Path
import numpy as np
import pandas as pd
import re

In [18]:
def create_atlas_dictionary(path_to_atlas_desc_file):

    #atlas_file = Path("/home-local/kimm58/AtlasToDiffusionReg/data/JHU_MNI_SS_WMPM_Type-III_SlicerLUT.txt")
    atlas_doc = np.loadtxt(path_to_atlas_desc_file, dtype=str, delimiter='\n')
    atlas_dict = dict()
    for label in atlas_doc[1:]:   #assuming that the labelmap desciption must have a header
        splits = label.split(" ")
        atlas_dict[int(splits[0])] = splits[1]
    return atlas_dict

#get list of atlas names from the inputs folder
def get_atlas_names(path):
    atlases = []
    for f in [x.name for x in path.glob('*')]:
        name_match = re.search("^(Atlas.*).nii.*$", f, re.IGNORECASE)
        if name_match:
            atlases.append(name_match.group(1))
    return atlases

#get list of label names from the inputs folder
def get_label_file_names(path, atlas_names):
    labels = []
    for atlas_name in atlas_names:
        atlas_match = re.search("^Atlas_(.*)$",atlas_name, re.IGNORECASE)
        atlas = atlas_match.group(1)
        label_match = re.compile("^(Labels.*{}.txt)$".format(atlas))
        for f in [x.name for x in path.glob('*')]:
            #print(f)
            mat = label_match.match(f)
            #print(mat)
            if mat:
                labels.append(f)
                print(atlas)
                print(f)
    return labels

def calc_scalars(atlas_name, roi, key, label, df, scalars, scalar_prefixes, idx):
    row = [atlas_name, roi]
    numPixels = np.sum(label == key)
    if numPixels == 0:
        row = row + [np.nan]*len(scalars)*2   #*2 for mean and std dev
        df.loc[idx] = row
        return
    for scalar,scalar_prefix in zip(scalars, scalar_prefixes):
        if np.sum(np.isnan(scalar[label == key])) > 0.5 * numPixels:
            row.append(np.nan) #mean
            row.append(np.nan)  #std
        else:
            row.append(np.nanmean(scalar[label == key]))
            row.append(np.nanstd(scalar[label == key]))
    df.loc[idx] = row
    return


In [19]:
#ATLASES need to be called:
    #"Atlas_<NAME>.nii.gz" where <NAME> is the name of the atlas
#Corresponding labels need to be called:
    #"Labels_<NAME>.txt"


#file_name = [x for x in Path("/INPUTS").glob('*.bval*')][0].name
inp = Path("/home-local/kimm58/AtlasToDiffusionReg/data/inputs")
out = Path("/home-local/kimm58/AtlasToDiffusionReg/data/outputs")
#inp = Path("/INPUTS/")
#out = Path("/OUTPUTS/")
file_name = [x for x in inp.glob('*.bval*')][0].name
name_match = re.search("^(.*).bval$", file_name)
name = name_match.group(1)

#inputs/intermediates
fa = out/("{}%{}.nii.gz".format(name, 'fa'))
md = out/("{}%{}.nii.gz".format(name, 'md'))
ad = out/("{}%{}.nii.gz".format(name, 'ad'))
rd = out/("{}%{}.nii.gz".format(name, 'rd'))

#GAMEPLAN
    #loop through atlases
        #loop through each region of each atlas
            #loop through each scalar
                #in this loop, append a new row to the dataframe

#preloading all the scalar files
scalar_files = [fa, md, ad, rd]
scalar_niftis = [nib.load(x) for x in scalar_files]
scalars = [x.get_fdata() for x in scalar_niftis]
scalar_prefixes = ['fa','md','ad','rd']

#setting up df
cols = ["Atlas Name", "ROI NAME"]
for p in scalar_prefixes:
    cols.append(p+'-mean')
    cols.append(p+'-std')
df = pd.DataFrame(columns=[cols])

#get atlas names
atlas_names = get_atlas_names(inp)
#get the atlases in a list
atlases = []
atlas_dicts = []
for atlas_name in atlas_names:
    #labelmap/atlas
    label_file = out/("{}%{}.nii.gz".format(name, atlas_name))
    label_nifti = nib.load(label_file)
    atlases.append(label_nifti)
    #dictionary for the labels

#set up atlas dictionaries
label_names = get_label_file_names(inp, atlas_names)
atlas_dicts = []
for label_name in label_names:
    dictionary_file = inp/(label_name)
    atlas_dict = create_atlas_dictionary(dictionary_file)
    atlas_dicts.append(atlas_dict)
    #print(atlas_dict)
    #print('x')

#actually calculating the rois
idx =0
for atlas_name,label_nifti,atlas_dict in zip(atlas_names, atlases, atlas_dicts):
    label = label_nifti.get_fdata()
    #function
    for key,roi in atlas_dict.items():
        calc_scalars(atlas_name, roi, key, label, df, scalars, scalar_prefixes, idx)
        idx += 1

df.to_csv(out/("{}%diffusionmetrics.csv".format(name)), index=False)


Empty DataFrame
Columns: [(Atlas Name,), (ROI NAME,), (fa-mean,), (fa-std,), (md-mean,), (md-std,), (ad-mean,), (ad-std,), (rd-mean,), (rd-std,)]
Index: []
JHU_MNI_SS_WMPM_Type-I
Labels_JHU_MNI_SS_WMPM_Type-I.txt
JHU_MNI_SS_WMPM_Type-II
Labels_JHU_MNI_SS_WMPM_Type-II.txt
JHU_MNI_SS_WMPM_Type-III
Labels_JHU_MNI_SS_WMPM_Type-III.txt
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float

KeyboardInterrupt: 

In [21]:
#need to implement looping through atlases






# #calculate the scalars for each ROI
# for scalar_name,scalar_prefix in zip(scalar_files, scalar_prefixes): #loop through all the scalar niftis (FA, MD, etc.)

#     #scalar_file = Path("/OUTPUTS/{}%{}.nii.gz".format(name,scalar_prefix))  #the scalar nifti
#     scalar_file = out/("{}%{}.nii.gz".format(name,scalar_prefix))
#     assert scalar_file.exists(), "ERROR: scalar file {} does not exist".format(scalar_file)
#     scalar_nifti = nib.load(scalar_file)
#     label_file = out/("{}%Atlas_JHU_MNI_SS_WMPM_Type-I.nii.gz".format(name))
#     label_nifti = nib.load(label_file)



#     #########################################
#     #put this into its own function
#         #make 4 rows, one for each diffusion metric?
#         #arguments to pass: df, 
#     atlas_name = "JHU_MNI_SS_WMPM_Type-I" #pass as argument
#     label = label_nifti.get_fdata() #pass as argument
#     scalar = scalar_nifti.get_fdata() #pass as argument
    
#     idx = 0     #perhaps pass this as an argument as well, have it increment outside the function every time the funtion gets called
#     lst = []
#     for key,val in atlas_dict.items():
#         std_dev_name = "{}-{} {}-{}-std".format(atlas_name, key, val, scalar_prefix)
#         mean_name = "{}-{} {}-{}-mean".format(atlas_name, key, val, scalar_prefix)
#         numPixels = np.sum(label == key)
#         if numPixels == 0:      #if there are no voxels, make it nan
#             #metric_dict[mean_name] = np.nan
#             lst.append(np.nan)
#             #metric_dict[std_dev_name] = np.nan
#             lst.append(np.nan)
#         else:
#             if np.sum(np.isnan(scalar[label == key])) > 0.5 * numPixels: #if over half the pixels are invalid, make it nan
#                 #metric_dict[mean_name] = np.nan
#                 lst.append(np.nan)
#                 #metric_dict[std_dev_name] = np.nan
#                 lst.append(np.nan)
#             else:
#                 #metric_dict[mean_name] = np.nanmean(scalar[label == key])
#                 lst.append(np.nanmean(scalar[label == key]))
#                 #metric_dict[std_dev_name] = np.nanstd(scalar[label == key])
#                 lst.append(np.nanstd(scalar[label == key]))
#         #here, need to insert the values into the df
#     print(lst)
#     print(df.head)
#     df.loc[idx] = lst

#     #df = pd.DataFrame.from_dict(metric_dict, index = [0])
#     print(df.head)

#     #will eventually need to loop through the atlases in the input/outputs dir
#     #for atlas_dict, label_nii


f = '/nfs2/kimm58/EVE3AtlasInputs/T1_TICV_seg.txt'
atlas_doc = np.loadtxt(f, dtype=str)
atlas_dict = dict()
for label in atlas_doc[1:]:
    #print(label)
    atlas_dict[int(label[0])] = label[1]
    #print(label[0], label[1])
    #break
#print(atlas_dict)

### Revised Functions

##### calc_metrics

In [4]:
def create_atlas_dictionary(path_to_atlas_desc_file):
    #atlas_file = Path("/home-local/kimm58/AtlasToDiffusionReg/data/JHU_MNI_SS_WMPM_Type-III_SlicerLUT.txt")
    atlas_doc = np.loadtxt(path_to_atlas_desc_file, dtype=str)
    atlas_dict = dict()
    for label in atlas_doc[1:]:   #assuming that the labelmap desciption must have a header
        atlas_dict[int(label[0])] = label[1]
    return atlas_dict

### main body

#add in segmentation map if it exists as well

atlas_names = []
atlases= []
#path_to_seg_labels = inp/("T1_seg.txt")
path_to_seg_labels = Path("/nfs2/kimm58/EVE3AtlasInputs/T1_TICV_seg.txt")
seg = path_to_seg_labels.parent/('template.nii.gz')
if seg.exists() and path_to_seg_labels.exists():
    atlas_names.append("T1_segmentation")   #add the segmentation to the list of atlases/labelmap names
    seg_nifti = nib.load(seg)
    atlases.append(seg_nifti)           #add the nifti to the list of labelmaps
    label_doc = np.loadtxt(path_to_seg_labels, dtype=str)                                                               ###### LINE CHANGED
    label_dict = dict()
    for label in label_doc[1:]:
        #splits = re.split(r'[,\s]+', label)  #splits by comma or space #.split(',')                                    ###### REMOVE LINE
        label_dict[int(label[0])] = label[1]                                                                            ###### LINE CHANGED
    atlas_dicts.append(label_dict)                  #add the label dicitonary to the list of labelmap dictionaries